In [1]:
from __future__ import print_function

In [2]:
import numpy as np
print(np.__version__)

1.10.4


In [3]:
import pandas as pd
print(pd.__version__)

0.18.1


In [4]:
import scipy as sp
print(sp.__version__)
from scipy.stats import pearsonr, spearmanr, kendalltau

0.17.1


In [5]:
import oddt
print(oddt.__version__)
from oddt.scoring.functions import rfscore
from oddt.scoring.models.regressors import randomforest
from oddt.metrics import enrichment_factor, roc_auc, roc_log_auc

0.1.15-54-g512d0bc


In [6]:
import sklearn
print(sklearn.__version__)
from sklearn.metrics import accuracy_score, precision_score, mean_squared_error, recall_score
from sklearn.utils import compute_sample_weight

0.17.1


In [7]:
from compiledtrees import CompiledRegressionPredictor

In [8]:
dude_ids = ['aa2ar', 'abl1', 'ace', 'aces', 'ada', 'ada17', 'adrb1', 'adrb2', 'akt1', 'akt2', 'aldr', 'ampc', 'andr', 'aofb', 'bace1', 'braf', 'cah2', 'casp3', 'cdk2', 'comt', 'cp2c9', 'cp3a4', 'csf1r', 'cxcr4', 'def', 'dhi1', 'dpp4', 'drd3', 'dyr', 'egfr', 'esr1', 'esr2', 'fa10', 'fa7', 'fabp4', 'fak1', 'fgfr1', 'fkb1a', 'fnta', 'fpps', 'gcr', 'glcm', 'gria2', 'grik1', 'hdac2', 'hdac8', 'hivint', 'hivpr', 'hivrt', 'hmdh', 'hs90a', 'hxk4', 'igf1r', 'inha', 'ital', 'jak2', 'kif11', 'kit', 'kith', 'kpcb', 'lck', 'lkha4', 'mapk2', 'mcr', 'met', 'mk01', 'mk10', 'mk14', 'mmp13', 'mp2k1', 'nos1', 'nram', 'pa2ga', 'parp1', 'pde5a', 'pgh1', 'pgh2', 'plk1', 'pnph', 'ppara', 'ppard', 'pparg', 'prgr', 'ptn1', 'pur2', 'pygm', 'pyrd', 'reni', 'rock1', 'rxra', 'sahh', 'src', 'tgfr1', 'thb', 'thrb', 'try1', 'tryb1', 'tysy', 'urok', 'vgfr2', 'wee1', 'xiap']

In [9]:
%%time
out = []
for engine, v, trees in [(e, v, trees) for e in ['dude', 'dock', 'vina'] for v in [1,2,3] for trees in [500]]:#range(100,2001,100)]:
# for engine, v, trees in [(e, v, trees) for e in ['vina'] for v in [3] for trees in [500]]:#range(100,2001,100)]:
    
    train_full = []
    
    if v == 1:
        col_range = range(1,37)
        np_type = np.uint16
    elif v == 2:
        col_range = range(1,217)
        np_type = np.uint16
    elif v == 3:
        col_range = range(1,43)   
        np_type = np.float16
    
    act_cutoff = 6.
    inactive_cutoff = 5.95

    for dude_id in dude_ids:
        # trap errors on reading
        try:
            actives_full = pd.read_csv('head1_full/%s/%s/%s_desc_v%i.csv.gz' % (dude_id, engine, 'actives', v), dtype={i: np_type for i in col_range})
            decoys_full = pd.read_csv('head1_full/%s/%s/%s_desc_v%i.csv.gz' % (dude_id, engine, 'decoys', v), dtype={i: np_type for i in col_range})
        except:
            continue
        decoys_full['act'] = inactive_cutoff if act_cutoff > 10 else 10**(9-inactive_cutoff)
        
        # generate one big table for dude_id
        train = pd.concat((actives_full, decoys_full))
        
        # normalize
        if act_cutoff >= 10:
            train['act'] = np.clip(train['act'], 1e-9, inactive_cutoff)
        else:
            train['act'] = np.clip(-np.log10(np.clip(train['act'], 1e-9, 1e9) * 1e-9), 0 , 15)
        # binary
        if act_cutoff >= 10:
            train['act_bin'] = train['act'] < act_cutoff
        else:
            train['act_bin'] = train['act'] > act_cutoff

        
        if len(train_full) == 0:
            train_full = train
        else:
            train_full = pd.concat((train_full, train))

    if v == 2:
        mtry = 50
    else:
        mtry = 15

    # Train Random Forest
    oddt.random_seed(0)
    rf = randomforest(n_estimators=trees, n_jobs=-1, verbose=1,
                            max_features=mtry, 
                            oob_score=True,
                            bootstrap = True,
                            random_state = 0,
                           )
    rf.fit(train_full[col_range].values,
           train_full['act'].values,
           sample_weight=compute_sample_weight('balanced', train_full['act_bin'].values))   
    rf.verbose = 0

    # Gather predictions
    train_full['pred'] = rf.oob_prediction_
    if act_cutoff > 10:
        train_full['pred_bin'] = train_full['pred'] < act_cutoff 
    else:
        train_full['pred_bin'] = train_full['pred'] > act_cutoff
    
    test_df = train_full.sort_values('pred', ascending=(act_cutoff >= 10))# log is descending

    d = {'engine': engine,
         'v': v,
         'trees': trees,
         
         'roc_auc': roc_auc(test_df['act_bin'], test_df['pred_bin'], ascending_score=False),# Binary is descending

         'ef0.1': enrichment_factor(test_df['act_bin'], test_df['pred_bin'], percentage=0.1),
         'ef1': enrichment_factor(test_df['act_bin'], test_df['pred_bin']),
         'ef2': enrichment_factor(test_df['act_bin'], test_df['pred_bin'], percentage=2),
         'ef5': enrichment_factor(test_df['act_bin'], test_df['pred_bin'], percentage=5),
         'ef10': enrichment_factor(test_df['act_bin'], test_df['pred_bin'], percentage=10),

         'rp': pearsonr(test_df['act'], test_df['pred'])[0],
         'rs': spearmanr(test_df['act'], test_df['pred'])[0],
         'rk': kendalltau(test_df['act'], test_df['pred'])[0],

         'rp_active': pearsonr(test_df['act'][test_df['act_bin']], test_df['pred'][test_df['act_bin']])[0],
         'rs_active': spearmanr(test_df['act'][test_df['act_bin']], test_df['pred'][test_df['act_bin']])[0],
         'rk_active': kendalltau(test_df['act'][test_df['act_bin']], test_df['pred'][test_df['act_bin']])[0],

         'rp_inactive': pearsonr(test_df['act'][~test_df['act_bin']], test_df['pred'][~test_df['act_bin']])[0],
         'rs_inactive': spearmanr(test_df['act'][~test_df['act_bin']], test_df['pred'][~test_df['act_bin']])[0],
         'rk_inactive': kendalltau(test_df['act'][~test_df['act_bin']], test_df['pred'][~test_df['act_bin']])[0],

         'mse': mean_squared_error(test_df['act'], test_df['pred']),
         'roc_log_auc': roc_log_auc(test_df['act_bin'], test_df['pred_bin'], ascending_score=False),# Binary is descending
         'precision': precision_score(test_df['act_bin'], test_df['pred_bin']),
         'accuracy': accuracy_score(test_df['act_bin'], test_df['pred_bin']),
         'recall': recall_score(test_df['act_bin'], test_df['pred_bin']),
        }
    print(d)
    
    out.append(d)
    
    r = rfscore.load(version=v)
    r.model = CompiledRegressionPredictor(rf, n_jobs=-1)
    r.score_title = 'RFScoreVS_v%i_%s' % (v, engine)
    pkl_file = 'RFScoreVS_v%i_%s.pickle' % (v,engine)
    r.save(pkl_file)

[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.7min finished


{'rs_inactive': 0.0078994276925536989, 'ef10': 6.631390864542062, 'rk_inactive': 0.0064541255446605439, 'rk_active': 0.18090721603973298, 'rp': 0.47391474234695885, 'rs': 0.16004379292960716, 'rp_active': 0.28927875179221163, 'mse': 0.052550697921816339, 'accuracy': 0.85428775349936326, 'engine': 'dude', 'rs_active': 0.26606187778568879, 'precision': 0.076577482106208017, 'trees': 500, 'roc_auc': 0.79370740938941364, 'roc_log_auc': 0.24308646293973404, 'ef1': 31.445210096721294, 'ef2': 20.802044237866514, 'ef5': 11.106028639128734, 'recall': 0.73111944295661491, 'rp_inactive': 0.019958151555354537, 'v': 1, 'ef0.1': 58.02107443999991, 'rk': 0.1303878889466468}


[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 17.2min finished


{'rs_inactive': 0.0078465061814315014, 'ef10': 7.436156142106129, 'rk_inactive': 0.0064082474586386459, 'rk_active': 0.19606032706589291, 'rp': 0.54525421990549217, 'rs': 0.17533394052567763, 'rp_active': 0.31485942935610955, 'mse': 0.04989076896040115, 'accuracy': 0.85112502926128408, 'engine': 'dude', 'rs_active': 0.28734859545006985, 'precision': 0.081412503552914739, 'trees': 500, 'roc_auc': 0.82865313658113282, 'roc_log_auc': 0.24955833784729434, 'ef1': 39.452691560669905, 'ef2': 25.446294217375172, 'ef5': 12.920879231794009, 'recall': 0.80543652919121589, 'rp_inactive': 0.019329849784086556, 'v': 2, 'ef0.1': 61.859391672184515, 'rk': 0.14281583451042326}


[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  7.8min finished


...

{'rs_inactive': 0.009496002683938725, 'ef10': 6.8824829966257175, 'rk_inactive': 0.0077581987271842017, 'rk_active': 0.18855667112018878, 'rp': 0.49993628125012346, 'rs': 0.17184784486455493, 'rp_active': 0.31410524506850324, 'mse': 0.052405994684015016, 'accuracy': 0.85626449567425433, 'engine': 'vina', 'rs_active': 0.2772833190982199, 'precision': 0.083859182995682499, 'trees': 500, 'roc_auc': 0.8054984846700699, 'roc_log_auc': 0.24770832724963257, 'ef1': 32.57859744904416, 'ef2': 21.583049021292123, 'ef5': 11.59838293222476, 'recall': 0.75294956566835214, 'rp_inactive': 0.022726785611511001, 'v': 1, 'ef0.1': 56.42110814765424, 'rk': 0.13998707767013771}


[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  9.6min finished


{'rs_inactive': 0.010433739344630977, 'ef10': 7.675294214942874, 'rk_inactive': 0.0085206911826491896, 'rk_active': 0.21178106365988228, 'rp': 0.56576059334030848, 'rs': 0.18675814567537186, 'rp_active': 0.34524676685701278, 'mse': 0.050021093613616938, 'accuracy': 0.85261892638809311, 'engine': 'vina', 'rs_active': 0.30946487108073406, 'precision': 0.088657033249856049, 'trees': 500, 'roc_auc': 0.84075352623677413, 'roc_log_auc': 0.25378816709395324, 'ef1': 39.51435138268469, 'ef2': 26.237390272917814, 'ef5': 13.38624008218429, 'recall': 0.8284714119019837, 'rp_inactive': 0.021510063656661339, 'v': 2, 'ef0.1': 58.68831685622817, 'rk': 0.15211170520008768}


[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  4.3min finished


{'rs_inactive': 0.01049604612220454, 'ef10': 7.552126486831461, 'rk_inactive': 0.0085734858628343895, 'rk_active': 0.19332825361767192, 'rp': 0.55308048649677877, 'rs': 0.18435684512858758, 'rp_active': 0.32102290735327005, 'mse': 0.049968593726952892, 'accuracy': 0.86032026023756136, 'engine': 'vina', 'rs_active': 0.28413493637684489, 'precision': 0.091761176556669349, 'trees': 500, 'roc_auc': 0.83705766702979711, 'roc_log_auc': 0.25907886659808255, 'ef1': 38.15961066106705, 'ef2': 25.339582106691967, 'ef5': 13.142500311196333, 'recall': 0.81297808894074941, 'rp_inactive': 0.022040515371892562, 'v': 3, 'ef0.1': 58.29965250618692, 'rk': 0.15016020084899706}
CPU times: user 1d 12h 3min 11s, sys: 8min 51s, total: 1d 12h 12min 2s
Wall time: 4h 37min 48s


In [10]:
pd.DataFrame(out).to_csv('dude_final_sf.csv')

In [11]:
g = pd.DataFrame(out).groupby(['engine', 'v', 'trees']).mean()
g

accuracy      ef0.1        ef1      ef10        ef2  \
engine v trees                                                        
dock   1 500    0.900033  71.104521  27.412174  6.091570  18.078893   
       2 500    0.897485  83.772036  33.492457  6.682957  21.364282   
       3 500    0.901311  82.080014  32.751985  6.664494  21.068068   
dude   1 500    0.854288  58.021074  31.445210  6.631391  20.802044   
       2 500    0.851125  61.859392  39.452692  7.436156  25.446294   
       3 500    0.862815  61.546971  38.113648  7.402680  24.683039   
vina   1 500    0.856264  56.421108  32.578597  6.882483  21.583049   
       2 500    0.852619  58.688317  39.514351  7.675294  26.237390   
       3 500    0.860320  58.299653  38.159611  7.552126  25.339582   

                      ef5       mse  precision    recall        rk  rk_active  \
engine v trees                                                                  
dock   1 500     9.934102  0.039552   0.064117  0.612402  0.098635   0.163102   
       2 500    11.166842  0.039041   0.068331  0.677834  0.108516   0.183155   
       3 500    11.176285  0.038695   0.070599  0.671541  0.107305   0.172184   
dude   1 500    11.106029  0.052551   0.076577  0.731119  0.130388   0.180907   
       2 500    12.920879  0.049891   0.081413  0.805437  0.142816   0.196060   
       3 500    12.708418  0.049152   0.086586  0.791019  0.142140   0.189700   
vina   1 500    11.598383  0.052406   0.083859  0.752950  0.139987   0.188557   
       2 500    13.386240  0.050021   0.088657  0.828471  0.152112   0.211781   
       3 500    13.142500  0.049969   0.091761  0.812978  0.150160   0.193328   

                rk_inactive   roc_auc  roc_log_auc        rp  rp_active  \
engine v trees                                                            
dock   1 500       0.004484  0.757775     0.272468  0.370424   0.253168   
       2 500       0.005771  0.788850     0.279475  0.420988   0.277684   
       3 500       0.005350  0.787676     0.283291  0.421172   0.265466   
dude   1 500       0.006454  0.793707     0.243086  0.473915   0.289279   
       2 500       0.006408  0.828653     0.249558  0.545254   0.314859   
       3 500       0.006348  0.827502     0.258627  0.542714   0.308250   
vina   1 500       0.007758  0.805498     0.247708  0.499936   0.314105   
       2 500       0.008521  0.840754     0.253788  0.565761   0.345247   
       3 500       0.008573  0.837058     0.259079  0.553080   0.321023   

                rp_inactive        rs  rs_active  rs_inactive  
engine v trees                                                 
dock   1 500       0.012366  0.120973   0.241002     0.005488  
       2 500       0.008649  0.133132   0.270846     0.007067  
       3 500       0.012042  0.131633   0.254742     0.006550  
dude   1 500       0.019958  0.160044   0.266062     0.007899  
       2 500       0.019330  0.175334   0.287349     0.007847  
       3 500       0.021549  0.174489   0.278828     0.007772  
vina   1 500       0.022727  0.171848   0.277283     0.009496  
       2 500       0.021510  0.186758   0.309465     0.010434  
       3 500       0.022041  0.184357   0.284135     0.010496

In [19]:
from itertools import product

In [46]:
imp = rf.feature_importances_

In [31]:
idx = np.argsort(rf.feature_importances_, )[::-1]

In [41]:
labels = ['gauss1', 'gauss2', 'repulsion', 'hydrophobic', 'h-bonding', 'n_rotors'] + list(product([6, 7, 8, 9, 15, 16, 17, 35, 53], [6, 7, 8, 16]))

In [47]:
for n, i in enumerate(idx):
    print(n+1, labels[i], round(imp[i],3))

1 gauss2 0.065
2 gauss1 0.065
3 h-bonding 0.065
4 repulsion 0.056
5 (6, 8) 0.053
6 (6, 16) 0.053
7 hydrophobic 0.052
8 (6, 6) 0.049
9 (7, 8) 0.047
10 (6, 7) 0.046
11 (7, 7) 0.044
12 (7, 6) 0.043
13 (8, 6) 0.039
14 (8, 7) 0.038
15 n_rotors 0.037
16 (8, 8) 0.034
17 (7, 16) 0.029
18 (8, 16) 0.027
19 (16, 6) 0.022
20 (16, 7) 0.019
21 (16, 8) 0.017
22 (9, 6) 0.012
23 (9, 7) 0.011
24 (17, 6) 0.01
25 (17, 8) 0.01
26 (9, 8) 0.01
27 (17, 7) 0.009
28 (16, 16) 0.008
29 (9, 16) 0.008
30 (17, 16) 0.007
31 (35, 7) 0.003
32 (35, 6) 0.003
33 (35, 8) 0.003
34 (35, 16) 0.001
35 (15, 8) 0.001
36 (15, 7) 0.001
37 (15, 6) 0.001
38 (53, 6) 0.001
39 (53, 8) 0.001
40 (53, 16) 0.001
41 (53, 7) 0.001
42 (15, 16) 0.0
